# Metody kryptografii w analizie danych

# Algorytm RSA

Obecnie najpopularniejszy algorytm kryptografi asymetrycznej. Opublikowany w 1977 roku w *Scientific American's Mathematical Games* przez Rona Rivesta, Adiego Shamira i Leonarda Adlemana. Opiera się na problemie faktoryzacji liczb.

W uproszczonej wersji algorytm RSA wygląda następująco:

### Generowanie kluczy

- Wybieramy dwie losowe liczby pierwsze $p$ i $q$
- Obliczamy $n=pq$
- Obliczamy funkcję Eulera $\phi(n)=(p-1)(q-1)$
- Wybieramy $e\in\{2,...,\phi(n)-1\}$ względnie pierwszą z $\phi(n)$
- Obliczamy $d=e^{-1}\mod \phi(n)$
- Klucz publiczny to para $(n,e)$ a klucz prywatny to para $(n,d)$.


### Szyfrowanie
Mamy dany klucz publiczny $(n,e)$. Wiadomość szyfrowana jest liczbą $m\in [0,n)$.
- Obliczamy szyfrogram $$c=m^e\mod n.$$

Teoretycznie może się zdarzyć, że szyfrogram $c$ jest taki sam, jak szyfrowana wiadomość $m$.

### Deszyfrowanie
- Obliczamy odszyfrowaną wiadomość $$m=c^d=m^{ed}\mod n$$

## Ćwiczenie 1.

Zaimplementuj uproszczony algorytm RSA.

In [ ]:

from Crypto.Util.number import getStrongPrime 
def rsa(m):
    p = getStrongPrime(768)
    q = getStrongPrime(768)
    n = p*q
    eu = (p-1)*(q-1)
    e = getStrongPrime(640)
    d = round(e**(-1) % eu)
    return m**e % n, n, d

def decipher(c, n, d):
    return c**d

def gcd(p, q):
    while q != 0:
        p, q = q, p % q
    return p

def coprime(a, b):
    return gcd(a, b) == 1

c = rsa(231412)
print(c)
m = decipher(c)
print(m)


OverflowError: int too large to convert to float

# Proste ataki na RSA

## Brute force

Najprostszym (ale też najmniej efektywnym i najbardziej bezsensownym) możliwym atakiem na RSA jest atak brute force:

#### Krok 1.
- Wersja 1: tablicujemy wszystkie liczby pierwsze mniejsze lub równe $\sqrt{n}$ do tablicy $T$.
- Wersja 2: pomijamy tablicowanie i w kroku 2. po prostu sprawdzamy wszystkie liczby naturalne mniejsze lub równe $\sqrt{n}$

#### Krok 2.
- Wersja 1: sprawdzamy podzielność $n$ przez każdą $p\in T$.
- Wersja 2: sprawdzamy podzielność $n$ przez każdą liczbę naturalną mniejszą lub równą $\sqrt{n}$

#### Krok 3.
Mając faktoryzację $pq=n$ obliczamy $\phi(n)$ oraz odtwarzamy $d=e^{-1}\mod \phi(n)$.

## Ćwiczenie 2.
Zaimplementuj atak brute force i przetestuj go na swojej implementacji RSA. Jeżeli atak zakończył się sukcesem - popraw implementację kryptosystemu. Jakie założenia powinny spełniać parametry kryptosystemu, aby atak się nie powiódł?

In [ ]:
# TYPE YOUR CODE BELOW


## Faktoryzacja Fermata

Faktoryzacja Fermata liczby nieparzystej $n$ opiera się na znalezieniu pary liczb $a,b$ takich, że $n=a^2-b^2$. Wtedy od razu otrzymujemy faktoryzację $$n=(a+b)(a-b).$$

Dla dowolnej złożonej liczby nieparzystej (tzn. liczby nieparzystej nie będącej liczbą pierwszą) znajdziemy takie liczby $a,b$. W szczególności, w przypadku $n=pq$ (jak w RSA) mamy $$a=\frac{p+q}{2},\qquad b=\frac{p-q}{2}.$$

#### Krok 0.
Sprawdzamy, czy $\sqrt{n}$ jest liczbą naturalną. Jeżeli tak - znaleźliśmy faktoryzację i koniec algorytmu.
#### Krok 1.
Definiujemy zmienne pomocnicze
$$a = \left\lceil\sqrt{n}\right\rceil,\qquad
b^2 = r^2 - n.$$
#### Krok 2.
Jeżeli $\sqrt{b^2}$ jest liczbą całkowitą, to kończymy algorytm i zwracamy $a$ oraz $b=\sqrt{b^2}$. Jeżeli nie, to aktualizujemy zmienne:
$$a=a + 1,\qquad b^2=a^2 - n.$$


## Ćwiczenie 3.
Zaimplementuj atak z użyciem faktoryzacji Fermata i przetestuj go na swojej implementacji RSA. Jeżeli atak zakończył się sukcesem - popraw implementację kryptosystemu. Jakie założenia powinny spełniać parametry kryptosystemu, aby atak się nie powiódł?

In [ ]:
# TYPE YOUR CODE BELOW


# Ataki nie wykorzystujące faktoryzacji

## Atak naiwny

Jeżeli wykładnik $e$ jest niewielki oraz dla wiadomości $m$ wartość $m^e<n$, to $c=m^e\mod n=m^e$. Możemy zatem odzyskać $m$ obliczając $m=\sqrt[e]{c}$.

## Ćwiczenie 3.
Sprawdź, czy Twoja implementacja RSA jest podatna na powyższy atak wykonując co najmniej 1000 prób. Jeżeli którakolwiek zakończyła się powodzeniem - popraw implementację. Jakie założenia powinny spełniać parametry kryptosystemu, żeby atak nie miał szans powodzenia?

In [ ]:
# TYPE YOUR CODE BELOW


## Atak Wienera

Atak Wienera odtwarza klucz prywatny $d$ z klucza publicznego $(n,e)$ bez konieczności faktoryzacji $n$.

#### Krok 1.
Przeksztacamy ułamek $\frac{e}{n}$ na ułamek łańcuchowy postaci $${\displaystyle x=a_{0}+{\cfrac {1}{a_{1}+{\cfrac {1}{a_{2}+{\cfrac {1}{a_{3}+{\cfrac {1}{a_{4}+\ddots \,\frac{1}{a_k}}}}}}}}}}$$o standardowej reprezentacji $[a_0;a_1,a_2,...,a_{k−2},a_{k−1},a_k]$.

#### Krok 2.
Dla każdego reduktu ułamka łańcuchowego, tzn. ułamka łańcuchowego postaci $[a_0]$, $[a_0;a_1]$, $[a_0;a_1,a_2]$,...,$[a_0;a_1,a_2,...,a_{k−2},a_{k−1},a_k]$ sprawdzamy, czy jest on rozwinięciem ułamka $\frac{s}{d}$ w ułamek łańcuchowy dla pewnej stałej $s$:
- niech $l$ oznacza licznik a $m$ - mianownik reduktu
- sprawdź, czy mianownik $m$ jest nieparzysty. Jeżeli nie - przejdź do sprawdzania kolejnego reduktu.
- sprawdź, czy $em=1\mod l$. Jeżeli nie - przejdź do sprawdzania kolejnego reduktu.
- zdefiniuj wielomian pomocniczy $$Q(x)=x^2-(n-\frac{em-1}{l}+1)x+n.$$Jeżeli pierwiastki tego wielomianu są liczbami całkowitymi, to aktualny mianownik $m$ reduktu jest szukanym kluczem prywatnym $d$.


Atak Wienera niekoniecznie musi zakończyć się sukcesem - jeżeli sprawdzimy wszystkie redukty i żaden nie jest rozwinięciem $\frac{s}{d}$, to dany zestaw parametrów RSA jest odporny na ten atak.

## Ćwiczenie 4.

Sprawdź, czy Twoja implementacja RSA jest podatna na atak Wienera wykonując co najmniej 100 prób. Jeżeli którakolwiek zakończyła się powodzeniem - popraw implementację. Jakie założenia powinny spełniać parametry kryptosystemu, żeby atak nie miał szans powodzenia?

In [ ]:
# TYPE YOUR CODE BELOW
